In [1]:
import numpy as np
import sys
sys.path.append("LRP_path")

from innvestigator import InnvestigateModel
from inverter_util import Flatten
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
import argparse
import torchvision
import torchvision.transforms as transforms
import torch.utils.data as Data
from models.VGG_16 import VGG16
# from models.vgg import vgg16_bn

from models.sa_models import ConvnetMnist, ConvnetCifar
import pickle
from get_a_single_path import getPath

os.environ["CUDA_VISIBLE_DEVICES"] = "7"

batch_size = 32
dataset = {}
dataloader = {}
transform_test = transforms.Compose([
    transforms.ToTensor(),
])
mnist_data = torchvision.datasets.MNIST(
    root='~/.torch', train=False, download=False, transform=transform_test)
mnist_loader = Data.DataLoader(dataset=mnist_data, batch_size=batch_size, shuffle=False)

cifar10_data = torchvision.datasets.CIFAR10(
        root = './data/cifar-10',
        train = True,
        transform = transform_test,
        download = False)
cifar10_loader = Data.DataLoader(dataset=cifar10_data, batch_size=batch_size, shuffle=False)

dataset["mnist"] = mnist_data
dataset["cifar10"] = cifar10_data

dataloader["mnist"] = mnist_loader
dataloader["cifar10"] = cifar10_loader

models = {}

model_convmnist = ConvnetMnist() 
model_convmnist.load_state_dict(torch.load("./trained_models/mnist_mixup_acc_99.28_ckpt.pth")["net"])

model_convcifar = ConvnetCifar() 
model_convcifar.load_state_dict(torch.load("./trained_models/cifar_mixup_acc_90.36_ckpt.pth")["net"])

model_vgg = VGG16(num_classes=10)
model_path = "./trained_models/model_vgg_cifar/vgg_seed32_dropout.pkl"
checkpoint = torch.load(model_path)
model_vgg.load_state_dict(checkpoint)

models["convmnist"] = model_convmnist
models["convcifar10"] = model_convcifar
models["vgg"] = model_vgg

# load adv data

In [ ]:
## ConvnetMnist
import torch
from dataloader_adv_test import DatasetAdv
from neuron_coverage import Coverager
# convmnist 0.8, 4, 0.8
# convcifar 0.7 7 0.9
# vgg 0.9 7 0.9

num_classes = 10

bucket_m = 100
# dic = {
#     "convcifar10": "1l0n0kqPbqcgEStCy76K9wiiO-Y4dWB35",
#     "convmnist": "199b0k5M7OUliRzVGepMhYDRwdZ1ofuHe",
#     "vgg": "1wpf8uNfTStBn7NkS0omhVfv2OgIlAqbU"
# }
dic = {
    "convcifar10": "1CSEGygqLyPiI02B5yn0pkqySf7-pndEl",
    "convcifar10_nature": "1CSEGygqLyPiI02B5yn0pkqySf7-pndEl",
    "convmnist": "1ukogsUcDEr8lE-JsXr2qTd5DMEsHBWha",
    "vgg": "162dCbZ-ksEh08s-njCSTDgj11uGMVwbF"
}
model_names = ["convmnist", "convcifar10"]
attacks = ["pgd", "cw"]
modes = ["mix"]

for model_name in ["vgg"]:
    for mode in ["mix"]:
        if model_name == "convmnist":
            cluster_threshold = 0.8
            num_cluster = 4
            single_threshold = 0.8
        elif model_name == "convcifar10":
            cluster_threshold = 0.9
            num_cluster = 7
            single_threshold = 0.7
        elif model_name == "vgg":
            cluster_threshold = 0.9
            num_cluster = 7
            single_threshold = 0.9
        print(model_name, mode)
        intra = {}
        layer_intra = {}

        test_set = DatasetAdv(dic[model_name])
        
        fetch_func = lambda x:x["your_adv"]

        
        for index, datax in enumerate(test_set):
            covered_10 = set()
            covered_100 = set() 
#                 print("index", index)
#             print(datax["key"],datax["your_adv"].shape,datax["your_label"].shape,"lbl")
            keys = datax["key"]
            print("keys:", keys)
#             x_test =torch.from_numpy(datax["your_adv"]) 
#             y_test = torch.from_numpy(datax["your_label"] )
            x_test = datax["your_adv"]
            y_test = datax["your_label"] 
#             print(x_test.shape,type(x_test))
#             print(y_test.shape,type(y_test))
            test_loader1=torch.torch.utils.data.DataLoader(
                    torch.utils.data.TensorDataset(x_test, y_test),
                    batch_size=x_test.shape[0])
#             if index < 10:
            for step, (x, y) in enumerate(test_loader1):
                print("step", step)
                x = x.cuda()
                
                models[model_name] = models[model_name].cuda()
                cover = Coverager(models[model_name], model_name, cluster_threshold, num_classes=num_classes, num_cluster=num_cluster)
                covered1, total1 = cover.Intra_NPC(x, y, bucket_m, single_threshold, mode=mode, simi_soft=False)
                covered2, total2 = cover.Layer_Intra_NPC(x, y, bucket_m, single_threshold, mode=mode, simi_soft=False)
                covered_10 = covered_10 | covered1
                covered_100 = covered_100 | covered2 
#                     print(cover.get_simi(x, y, bucket_m, single_threshold, mode=mode, simi_soft=False))
            intra[keys] = len(covered_10) / total1
            layer_intra[keys] = len(covered_100) / total2
            print(intra[keys])
            print(layer_intra[keys])
        save_filename_10 = "./coverage_results/{}_Our_10_{}".format(mode, dic[model_name])
        save_filename_100 = "./coverage_results/{}_Our_100_{}".format(mode, dic[model_name])
        np.save(save_filename_10, intra) 
        np.save(save_filename_100, layer_intra) 

vgg mix
Using downloaded and verified file: ./adv_data_162dCbZ-ksEh08s-njCSTDgj11uGMVwbF.pkl
dict_keys(['timestamp', 'attack', 'model', 'layer', 'regularization_weight', 'confidence', 'adversaries', 'pert_acc', 'orig_acc', 'attack_success_rate', 'neuron_coverage_000', 'neuron_coverage_020', 'neuron_coverage_050', 'neuron_coverage_075', 'inception_score', 'fid_score_64', 'fid_score_2048', 'output_impartiality'])
keys: manu_div4_attack:0:0:0.01
step 0
0.18471428571428572
0.14217582417582417
keys: manu_div4_attack:0:0:0.03
step 0
0.18985714285714286
0.1443956043956044
keys: manu_div4_attack:0:0:0.05
step 0


In [ ]:
from dataloader_adv_test import DatasetAdv
from neuron_coverage import Coverager

single_threshold = 0.8
num_classes = 10
num_cluster = 7
bucket_m = 100
dic = {
    "convcifar10": "1Gc5n0UNRmJ2v9WjWZJ9JstmIoP3cWiFw",
    "convmnist": "12DdlhFwJQJl3W2gNbxaL9z9rIqf60mrD",
}
model_names = ["convmnist", "convcifar10"]
attacks = ["pgd", "cw"]
modes = ["right", "wrong", "mix"]

for model_name in ["convcifar10", "convmnist"]:
    for mode in ["mix"]:
        print(model_name, mode)
        intra = {}
        layer_intra = {}

        test_set = DatasetAdv(dic[model_name])
        
        fetch_func = lambda x:x["your_adv"]

        
        for index, datax in enumerate(test_set):
            covered_10 = set()
            covered_100 = set() 
#                 print("index", index)
#             print(datax["key"],datax["your_adv"].shape,datax["your_label"].shape,"lbl")
            keys = datax["key"]
            print("keys:", keys)
#             x_test =torch.from_numpy(datax["your_adv"]) 
#             y_test = torch.from_numpy(datax["your_label"] )
            x_test = datax["your_adv"]
            y_test = datax["your_label"] 
#             print(x_test.shape,type(x_test))
#             print(y_test.shape,type(y_test))
            test_loader1=torch.torch.utils.data.DataLoader(
                    torch.utils.data.TensorDataset(x_test, y_test),
                    batch_size=x_test.shape[0])
#             if index < 10:
            for step, (x, y) in enumerate(test_loader1):
                print("step", step)
                x = x.cuda()
                if model_name == "convmnist":
                    cluster_threshold = 0.4
                else:
                    cluster_threshold = 0.9
                models[model_name] = models[model_name].cuda()
                cover = Coverager(models[model_name], model_name, cluster_threshold, num_classes=num_classes, num_cluster=num_cluster)
                covered1, total1 = cover.Intra_NPC(x, y, bucket_m, single_threshold, mode=mode, simi_soft=False)
                covered2, total2 = cover.Layer_Intra_NPC(x, y, bucket_m, single_threshold, mode=mode, simi_soft=False)
                covered_10 = covered_10 | covered1
                covered_100 = covered_100 | covered2 
#                     print(cover.get_simi(x, y, bucket_m, single_threshold, mode=mode, simi_soft=False))
            intra[keys] = len(covered_10) / total1
            layer_intra[keys] = len(covered_100) / total2
            print(intra[keys])
            print(layer_intra[keys])
        save_filename_10 = "./coverage_results/{}_Our_10_{}".format(mode, dic[model_name])
        save_filename_100 = "./coverage_results/{}_Our_100_{}".format(mode, dic[model_name])
        np.save(save_filename_10, intra) 
        np.save(save_filename_100, layer_intra) 

In [ ]:
cifar10_test_data = torchvision.datasets.CIFAR10(
        root = './data/cifar-10',
        train = False,
        transform = transform_test,
        download = False)
cifar10_test_loader = Data.DataLoader(dataset=cifar10_test_data, batch_size=batch_size, shuffle=False)
mnist_test_data = torchvision.datasets.MNIST(
    root='~/.torch', train=False, download=False, transform=transform_test)
mnist_test_loader = Data.DataLoader(dataset=mnist_test_data, batch_size=batch_size, shuffle=False)


from dataloader_adv_test import DatasetAdv
from neuron_coverage import Coverager

width_threshold = 0.4
single_threshold = 0.8
num_classes = 10
num_cluster = 1
bucket_m = 5
dic = {
    "convmnist_pgd": "1hc_aj908k7_Zs2L4TsaWYENG-GwtdJe2",
    "convmnist_cw": "1aRN20FXhxvWqsIQTdDSNwei_jYSx5S7a",
    "convcifar10_pgd": "1nhWO0VT131_9e5ubgzs343EhM9Ru0UvY",
    "convcifar10_cw": "1MrRngrHDuSm2fEd044mNktkZtDSM33U4",
    "vgg_pgd": "1Gm926_p5_bvhgfDdlQmmV9lUCmjsF5Ft",
    "vgg_cw": "1-X1d-qaYGpUacej9McI5DOq20dDOk9ia",
}
model_names = ["convmnist", "convcifar10", "vgg"]
attacks = ["pgd", "cw"]
modes = ["right", "wrong", "mix"]

for model_name in ["convmnist"]:
    for attack in ["cw"]:
        for mode in ["right", "wrong"]:
            print("mode:", mode)
            for step, (x, y) in enumerate(mnist_test_loader):
                if step >= 30:
                    break
                x = x.cuda()
                models[model_name] = models[model_name].cuda()
                cover = Coverager(models[model_name], model_name, width_threshold, num_classes=num_classes, num_cluster=num_cluster)
#                         intra[keys] = cover.Intra_NPC(x, y, bucket_m, mode=mode, simi_soft=True)
#                         layer_intra[keys] = cover.Layer_Intra_NPC(x, y, bucket_m, mode=mode, simi_soft=True)
                print(cover.get_simi(x, y, bucket_m, single_threshold=0.8, mode=mode, simi_soft=True))

In [5]:
cluster_threshold = 0.4
single_threshold = 0.8
num_classes = 10
num_cluster = 7
bucket_m = 100
dic = {
    "convcifar10": "1Gc5n0UNRmJ2v9WjWZJ9JstmIoP3cWiFw",
    "convmnist": "12DdlhFwJQJl3W2gNbxaL9z9rIqf60mrD",
}
model_names = ["convmnist", "convcifar10"]
attacks = ["pgd", "cw"]
modes = ["right", "wrong", "mix"]

for model_name in ["convcifar10", "convmnist"]:
    for mode in ["mix"]:
        print(model_name, mode)
        intra = {}
        layer_intra = {}

        test_set = DatasetAdv(dic[model_name])
        
        fetch_func = lambda x:x["your_adv"]

        
        for index, datax in enumerate(test_set):
            covered_10 = set()
            covered_100 = set() 
#                 print("index", index)
#             print(datax["key"],datax["your_adv"].shape,datax["your_label"].shape,"lbl")
            keys = datax["key"]
            print("keys:", keys)
#             x_test =torch.from_numpy(datax["your_adv"]) 
#             y_test = torch.from_numpy(datax["your_label"] )
            x_test = datax["your_adv"]
            y_test = datax["your_label"] 
#             print(x_test.shape,type(x_test))
#             print(y_test.shape,type(y_test))
            test_loader1=torch.torch.utils.data.DataLoader(
                    torch.utils.data.TensorDataset(x_test, y_test),
                    batch_size=x_test.shape[0])
#             if index < 10:
            for step, (x, y) in enumerate(test_loader1):
                print("step", step)
                x = x.cuda()
                if model_name == "convmnist":
                    cluster_threshold = 0.4
                else:
                    cluster_threshold = 0.9
                models[model_name] = models[model_name].cuda()
                cover = Coverager(models[model_name], model_name, cluster_threshold, num_classes=num_classes, num_cluster=num_cluster)
                covered1, total1 = cover.Intra_NPC(x, y, bucket_m, single_threshold, mode=mode, simi_soft=False)
                covered2, total2 = cover.Layer_Intra_NPC(x, y, bucket_m, single_threshold, mode=mode, simi_soft=False)
                covered_10 = covered_10 | covered1
                covered_100 = covered_100 | covered2 
#                     print(cover.get_simi(x, y, bucket_m, single_threshold, mode=mode, simi_soft=False))
            intra[keys] = len(covered_10) / total1
            layer_intra[keys] = len(covered_100) / total2
            print(intra[keys])
            print(layer_intra[keys])
        save_filename_10 = "./coverage_results/{}_Our_10_{}".format(mode, dic[model_name])
        save_filename_100 = "./coverage_results/{}_Our_100_{}".format(mode, dic[model_name])
        np.save(save_filename_10, intra) 
        np.save(save_filename_100, layer_intra) 

convcifar10 mix
Using downloaded and verified file: ./adv_data_1Gc5n0UNRmJ2v9WjWZJ9JstmIoP3cWiFw.pkl
dict_keys(['timestamp', 'attack', 'model', 'layer', 'regularization_weight', 'confidence', 'adversaries', 'pert_acc', 'orig_acc', 'attack_success_rate', 'neuron_coverage_000', 'neuron_coverage_020', 'neuron_coverage_050', 'neuron_coverage_075', 'inception_score', 'fid_score_64', 'fid_score_2048', 'output_impartiality'])
keys: manu_div4_attack:0:0:0
step 0
0.182
0.19951785714285714
keys: manu_div4_attack:0:0:4
step 0
0.19042857142857142
0.20235714285714285
keys: manu_div4_attack:0:0:8
step 0
0.19714285714285715
0.20491071428571428
keys: manu_div4_attack:0:0:12
step 0
0.2
0.20760714285714285
keys: manu_div4_attack:0:0:16
step 0
0.2037142857142857
0.2095357142857143
keys: manu_div4_attack:0:0:20
step 0
0.20814285714285713
0.21182142857142858
keys: manu_div4_attack:0:0:24
step 0


KeyboardInterrupt: 

# Get the path: test of a single input

In [3]:
images = dataloader["mnist"]
model1 = models["convmnist"]
width_threshold = 0.8

for i, (data, target) in enumerate(images):
    data1 = data
    path = getPath(data, model1, width_threshold)
#     print(path)
    break

images = dataloader["cifar10"]
model2 = models["convcifar10"]
width_threshold = 0.8
for i, (data, target) in enumerate(images):
    data2 = data
    path = getPath(data, model2, width_threshold)
#     print(path)
    break

images = dataloader["cifar10"]
model3 = models["vgg"]
width_threshold = 0.8
for i, (data, target) in enumerate(images):
    data3 = data
    path = getPath(data, model3, width_threshold)
#     print(path)
    break

# test of neuron coverage

In [4]:
from neuron_coverage import Coverager

width_threshold = 0.8
num_classes = 10
num_cluster = 10
bucket_m = 10

from neuron_coverage import Coverager
cover = Coverager(model1, "convmnist", width_threshold, num_classes=num_classes, num_cluster=num_cluster)
X = dataset["mnist"][50][0].unsqueeze(0)
print(cover.Intra_NPC(X, bucket_m))
print(cover.Layer_Intra_NPC(X, bucket_m))

0.01
0.01


In [5]:
cover = Coverager(model2, "convcifar10", width_threshold, num_classes=num_classes, num_cluster=num_cluster)
X = dataset["cifar10"][50][0].unsqueeze(0)
print(cover.Intra_NPC(X, bucket_m))
print(cover.Layer_Intra_NPC(X, bucket_m))

0.01
0.01


In [6]:
cover = Coverager(model3, "vgg", width_threshold, num_classes=num_classes, num_cluster=num_cluster)
X = dataset["cifar10"][50][0].unsqueeze(0)
print(cover.Intra_NPC(X, bucket_m))
print(cover.Layer_Intra_NPC(X, bucket_m))

0.01
0.01


In [7]:
cover = Coverager(model1, "convmnist", width_threshold, num_classes=num_classes, num_cluster=num_cluster)
print(cover.Intra_NPC(data1, bucket_m))
print(cover.Layer_Intra_NPC(data1, bucket_m))

0.515
0.639


In [8]:
cover = Coverager(model2, "convcifar10", width_threshold, num_classes=num_classes, num_cluster=num_cluster)
print(cover.Intra_NPC(data2, bucket_m))
print(cover.Layer_Intra_NPC(data2, bucket_m))

0.386
0.550875


In [4]:
from neuron_coverage import Coverager

width_threshold = 0.8
num_classes = 10
num_cluster = 10
bucket_m = 10

cover = Coverager(model3, "vgg", width_threshold, num_classes=num_classes, num_cluster=num_cluster)
print(cover.Intra_NPC(data3, bucket_m))
print(cover.Layer_Intra_NPC(data3, bucket_m))

0.376
0.5511538461538461


In [6]:
a = [1, 2, 3]
np.mean(a)

2.0